In [1]:
import json
import os
import pandas as pd
import ray

from pprint import pprint 

from env import QdTreeEnv
from qdtree import Workload
from qdtree.schema import ensure_data_schema
from policy import QdTreePolicy

from ray.rllib.env.env_context import EnvContext
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.policy.sample_batch import DEFAULT_POLICY_ID
from ray import air, tune


2024-05-19 18:49:53,550	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-19 18:49:56,275	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Parameters

In [2]:
# We use sf = 1 for TPC-H. The paper uses sf = 1000
BASE_PATH = "../data/tpc-h/"
# The sampling rate in the paper is 0.1
SAMPLING_RATE = 0.01
# The min block size in the paper is 100K, we use 100 to scale down according to the sf
MIN_BLOCK_SIZE = 100 * SAMPLING_RATE

# Load the workload

In [3]:
WORKLOAD_PATH = os.path.join(BASE_PATH, "workload.json")
with open(WORKLOAD_PATH, "r") as f:
    workload = Workload(json.load(f))

workload._queries

{'1.1.sql': l_shipdate <= 903247200,
 '1.5.sql': l_shipdate <= 905666400,
 '1.3.sql': l_shipdate <= 907048800,
 '1.2.sql': l_shipdate <= 902556000,
 '1.4.sql': l_shipdate <= 906357600,
 '3.1.sql':  and(o_orderdate < 796287600, l_shipdate > 796287600),
 '3.4.sql':  and(o_orderdate < 795250800, l_shipdate > 795250800),
 '3.2.sql':  and(o_orderdate < 795078000, l_shipdate > 795078000),
 '3.3.sql':  and(o_orderdate < 796460400, l_shipdate > 796460400),
 '3.5.sql':  and(o_orderdate < 796633200, l_shipdate > 796633200),
 '4.3.sql':  and(o_orderdate >= 865144800, o_orderdate < 873093600),
 '4.2.sql':  and(o_orderdate >= 783673200, o_orderdate < 791622000),
 '4.1.sql':  and(o_orderdate >= 854780400, o_orderdate < 862466400),
 '4.5.sql':  and(o_orderdate >= 875685600, o_orderdate < 883638000),
 '4.4.sql':  and(o_orderdate >= 794041200, o_orderdate < 801986400),
 '5.4.sql':  and(o_orderdate >= 725871600, o_orderdate < 757407600),
 '5.2.sql':  and(o_orderdate >= 725871600, o_orderdate < 757407600

# Load and sample data

In [4]:
DATA_PATH = os.path.join(BASE_PATH, "sf1/denormalized.parquet")
all_data = pd.read_parquet(DATA_PATH).sample(frac=0.1)
all_data.columns = all_data.columns.str.lower()
all_data = ensure_data_schema(all_data, workload.schema)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600122 entries, 1810670 to 496868
Data columns (total 35 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   l_orderkey        600122 non-null  int64  
 1   l_partkey         600122 non-null  int64  
 2   l_suppkey         600122 non-null  int64  
 3   l_linenumber      600122 non-null  int64  
 4   l_quantity        600122 non-null  float64
 5   l_extendedprice   600122 non-null  float64
 6   l_discount        600122 non-null  float64
 7   l_tax             600122 non-null  float64
 8   l_shipdate        600122 non-null  int64  
 9   l_commitdate      600122 non-null  int64  
 10  l_receiptdate     600122 non-null  int64  
 11  o_orderkey        600122 non-null  int64  
 12  o_custkey         600122 non-null  int64  
 13  o_totalprice      600122 non-null  float64
 14  o_orderdate       600122 non-null  int64  
 15  o_shippriority    600122 non-null  int64  
 16  c_custkey     

In [5]:
data = all_data.sample(frac=SAMPLING_RATE)
len(data)

6001

# Set up the environment

In [6]:
ray.init(local_mode=False, ignore_reinit_error=True,num_cpus=8)

2024-05-19 18:51:54,761	INFO worker.py:1749 -- Started a local Ray instance.


Python version:,3.10.14
Ray version:,2.22.0


In [7]:
env_config = {
    "workload": workload,
    "data": data,
    "min_leaf_size": MIN_BLOCK_SIZE,
}

config = (
    PPOConfig()
    .environment(
        QdTreeEnv,
        env_config=env_config,
    )
    .framework("torch")
    .rollouts(num_rollout_workers=2, batch_mode="complete_episodes")
    .training(
        model={
            "fcnet_hiddens": [512, 512],
            "fcnet_activation": "relu",
        }
    )
)

algo = config.build()

stop = {
    "training_iteration": 10,
    "timesteps_total": 40000,
}

2024-05-19 18:52:00,545	WARNING deprecation.py:50 -- DeprecationWarning: `rollouts` has been deprecated. Use `AlgorithmConfig.env_runners(..)` instead. This will raise an error in the future!
2024-05-19 18:52:00,547	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.env_runners(num_rollout_workers)` has been deprecated. Use `AlgorithmConfig.env_runners(num_env_runners)` instead. This will raise an error in the future!
/Users/umsaka/anaconda3/envs/qdtree/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:521: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/umsaka/anaconda3/envs/qdtree/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ra

(raylet) [2024-05-19 18:52:04,797 E 19039 28058725] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-05-19_18-51-52_110067_17226 is over 95% full, available space: 7771734016; capacity: 250790436864. Object creation will fail if spilling is required.
(raylet) [2024-05-19 18:52:14,853 E 19039 28058725] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-05-19_18-51-52_110067_17226 is over 95% full, available space: 6696931328; capacity: 250790436864. Object creation will fail if spilling is required.
(raylet) [2024-05-19 18:52:24,921 E 19039 28058725] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-05-19_18-51-52_110067_17226 is over 95% full, available space: 5637046272; capacity: 250790436864. Object creation will fail if spilling is required.
(raylet) [2024-05-19 18:52:34,996 E 19039 28058725] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-05-19_18-51-52_110067_17226 is over 95% full, available space: 4556693504; capacity: 250790436864. Object 

In [8]:
os.chdir('/Users/umsaka/Documents/DDCPS/qdtree/src')

In [10]:
tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(
        local_dir="/Users/umsaka/Documents/DDCPS/qdtree/src/results",
        stop=stop,
        checkpoint_config=air.CheckpointConfig(checkpoint_at_end=True),
    ),
)

result = tuner.fit()

2024-05-19 19:00:21,996	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-05-19 19:00:22,102	INFO tune.py:1007 -- Wrote the latest version of all result files and experiment state to '/Users/umsaka/Documents/DDCPS/qdtree/src/results/PPO_2024-05-19_18-52-40' in 0.2979s.
2024-05-19 19:00:22,359	INFO tune.py:1039 -- Total run time: 461.49 seconds (460.79 seconds for the tuning loop).


In [11]:
agent = config.build()
agent.restore(result.get_best_result().checkpoint) # type: ignore

env = QdTreeEnv(EnvContext(env_config, 0))
done = False
obs, _ = env.reset() 
step = 0
episode_reward = 0
while not done:
    action = agent.compute_single_action(obs, explore=False)
    obs, reward, done, _, info = env.step(action) # type: ignore
    episode_reward += reward
    # print(step, action, obs, reward, done, info)
    step += 1

pprint(env.qd_tree.blocks)
print(episode_reward)


/Users/umsaka/anaconda3/envs/qdtree/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:521: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/umsaka/anaconda3/envs/qdtree/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/Users/umsaka/anaconda3/envs/qdtree/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This AP

[{'c_acctbal': (-inf, inf),
  'c_custkey': (-inf, inf),
  'c_nationkey': (-inf, inf),
  'l_commitdate': (-inf, inf),
  'l_discount': (-inf, inf),
  'l_extendedprice': (-inf, inf),
  'l_linenumber': (-inf, inf),
  'l_orderkey': (-inf, inf),
  'l_partkey': (-inf, inf),
  'l_quantity': (-inf, inf),
  'l_receiptdate': (-inf, inf),
  'l_shipdate': (-inf, 788943600),
  'l_suppkey': (-inf, inf),
  'l_tax': (-inf, inf),
  'n_nationkey_cust': (-inf, inf),
  'n_nationkey_supp': (-inf, inf),
  'n_regionkey_cust': (-inf, inf),
  'n_regionkey_supp': (-inf, inf),
  'o_custkey': (-inf, inf),
  'o_orderdate': (-inf, 757407600),
  'o_orderkey': (-inf, inf),
  'o_shippriority': (-inf, inf),
  'o_totalprice': (-inf, inf),
  'p_partkey': (-inf, inf),
  'p_retailprice': (-inf, inf),
  'p_size': (-inf, inf),
  'ps_availqty': (-inf, inf),
  'ps_partkey': (-inf, inf),
  'ps_suppkey': (-inf, inf),
  'ps_supplycost': (-inf, inf),
  'r_regionkey_cust': (-inf, inf),
  'r_regionkey_supp': (-inf, inf),
  's_acctbal

In [ ]:
print(env.qd_tree)